In [48]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import nba_api
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go 
import plotly.express as px

fig_DIR = "./figs/team_leaders/"

In [24]:
def get_df(stats):
    df = stats.get_data_frames()[0]
    df["Team"] = df["TEAM_NAME"]
    cols = df.columns
    df.columns = cols.str.replace("RATING","RTG")
    return df

In [237]:
from nba_api.stats.endpoints import leaguedashteamstats
stats = leaguedashteamstats.LeagueDashTeamStats(measure_type_detailed_defense="Advanced")
df = get_df(stats)
df = df.rename(columns={"OFF_RTG_RANK":"O_RANK","DEF_RTG_RANK":"D_RANK"})

In [238]:
df.columns

Index(['TEAM_ID', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'E_OFF_RTG',
       'OFF_RTG', 'E_DEF_RTG', 'DEF_RTG', 'E_NET_RTG', 'NET_RTG', 'AST_PCT',
       'AST_TO', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT',
       'EFG_PCT', 'TS_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE',
       'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'O_RANK',
       'D_RANK', 'NET_RTG_RANK', 'AST_PCT_RANK', 'AST_TO_RANK',
       'AST_RATIO_RANK', 'OREB_PCT_RANK', 'DREB_PCT_RANK', 'REB_PCT_RANK',
       'TM_TOV_PCT_RANK', 'EFG_PCT_RANK', 'TS_PCT_RANK', 'PACE_RANK',
       'PIE_RANK', 'Team'],
      dtype='object')

In [235]:
df1 = df.sort_values(by="NET_RTG",ascending=False)
df1 = df1.reset_index(drop=True)
df1 = df1.reset_index(drop=False)
df1["#"] = df1["index"] +1
df2 = df1[["#","Team","OFF_RTG","O_RANK","DEF_RTG","D_RANK","NET_RTG"]]
df = df2

In [241]:
df

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,E_OFF_RTG,OFF_RTG,E_DEF_RTG,...,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK,Team
0,1610612737,Atlanta Hawks,6,4,2,0.667,288.0,116.8,119.3,110.6,...,10,5,11,5,13,10,5,4,7,Atlanta Hawks
1,1610612738,Boston Celtics,5,5,0,1.000,240.0,122.2,124.7,104.7,...,26,4,6,1,9,1,1,10,1,Boston Celtics
2,1610612751,Brooklyn Nets,6,3,3,0.500,288.0,113.1,115.4,113.1,...,7,16,2,8,8,5,10,18,11,Brooklyn Nets
3,1610612766,Charlotte Hornets,5,2,3,0.400,240.0,111.0,112.2,114.0,...,13,11,26,16,16,16,14,2,16,Charlotte Hornets
4,1610612741,Chicago Bulls,7,2,5,0.286,341.0,104.4,106.2,112.8,...,28,18,16,25,2,28,28,29,30,Chicago Bulls
5,1610612739,Cleveland Cavaliers,6,2,4,0.333,288.0,105.3,105.8,107.1,...,21,26,27,28,3,26,26,22,22,Cleveland Cavaliers
6,1610612742,Dallas Mavericks,5,4,1,0.800,240.0,118.5,119.8,113.3,...,24,21,22,21,6,3,2,15,9,Dallas Mavericks
7,1610612743,Denver Nuggets,7,6,1,0.857,336.0,113.7,117.1,104.8,...,1,9,14,9,23,4,8,27,2,Denver Nuggets
8,1610612765,Detroit Pistons,6,2,4,0.333,288.0,106.2,109.1,108.6,...,8,2,3,3,30,20,20,14,17,Detroit Pistons
9,1610612744,Golden State Warriors,6,5,1,0.833,288.0,114.6,116.5,106.4,...,4,6,23,10,20,9,9,17,6,Golden State Warriors


In [233]:
n_colors = 30
colors = px.colors.sample_colorscale("Tropic", [n/(n_colors -1) for n in range(n_colors)])
orank = df["O_RANK"]-1
orank = orank.astype(np.int32).to_numpy()
carray_o =  np.array(colors)[orank]
drank = df["D_RANK"]-1
drank = drank.astype(np.int32).to_numpy()
carray_d =  np.array(colors)[drank]
nrank = df["#"]-1
nrank = nrank.astype(np.int32).to_numpy()
carray_n =  np.array(colors)[nrank]
title = "NBA Net Rating Leaders 2023-24"
font = dict(color="black",family="Arial Black, monospace", size=11)
fig = go.Figure(data=[go.Table(
        columnwidth=[5,50,15,15,15,15,15],
        header=dict(values=list(df.columns),
                    fill_color='snow',
                    align=['center','left','center','center'],
                    font=dict(color='Black',family="Arial Black, monospace", size=12),
                    line_color="grey"
                    ),
        cells=dict(values=[df["#"],df["Team"],df["OFF_RTG"],df["O_RANK"],df["DEF_RTG"],
                           df["D_RANK"],df["NET_RTG"]],
                fill_color=["white","white",carray_o,"white",carray_d,"white",carray_n],
                align=['center','left','center','center'],
                height=23,
                line_color="grey",#lightgrey",
                font=font,
                ),
                # height=25
        ),
    ])
    # fig.update_layout(title_text=title)
fig.update_layout(title=dict(text=title,y=0.99,x=0.1,font=dict(size=15,family="Arial Black, monospace")))
fig.add_annotation(x=0.0, y=0.0,text="@SravanNBA",showarrow=False,xshift=1,yshift=5)
fig.add_annotation(x=0.9, y=0.0,text="Source: nba.com/stats",showarrow=False,xshift=1,yshift=5)
# fig.update_layout(title=dict(text=title,y=0.01,x=0.1,font=dict(size=15,family="Arial Black, monospace")))
tab_width = 550 
tab_height = 770
fig.update_layout(width=tab_width,height=tab_height,margin=dict(t=25,b=1,l=1,r=1))
# fig.update_layout(autosize=True)
fig.show()
fig.write_image(fig_DIR+f"NET_RTG.png", scale=3)